In [ ]:
!pip install langgraph

In [1]:
greetings_pool = [["salve", "hello"],
 ["ciao", "hi"],
 ["bounaserra", "good evening"],
 ["buongiorno", "good morning"]]

## State

We will define a graph.

Our state will be a dict.

We can access this from any graph node as state['keys'].

In [2]:
from typing import List, Dict, TypedDict
from pydantic import BaseModel, Field

In [3]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

## Nodes

Each node will simply modify the state.

In [4]:
def anna(state):
    state_dict = state["keys"]
    greetings = state_dict["greetings"]
    print("System Greetings")
    print(greetings)
    choice = random.randint(0,3)
    greetings = greetings_pool[choice]
    print("----Anna-----")
    print(greetings)
    return {
        "keys": {"greetings": greetings}
    }

In [5]:
import random
def matteo(state):
    print("------Matteo---------")
    state_dict = state["keys"]
    greetings = state_dict["greetings"]
    choice = random.randint(0,3)
    return_greetings = greetings_pool[choice]
    print(return_greetings)
    return {
        "keys": {"greetings": greetings, "return_greetings":return_greetings}
    }

## Edges

Each edge will choose which node to call next.

In [9]:
def greetings_grader(state):
    state_dict = state["keys"]
    greetings = state_dict["greetings"]
    return_greetings = state_dict["return_greetings"]
    grade = None
    if greetings == return_greetings:
        grade = "good greeter"
        print("DECISION-----GOOD GREETER----GOODBYE")
    else:
        grade = "bad greeter"
        print("DECISION-----BAD GREETER----TRY AGAIN")
    return grade

## Build and Compile Graph

In [10]:
from langgraph.graph import StateGraph, END

# Define a new graph
workflow = StateGraph(GraphState)

# Define the two nodes we will cycle between
workflow.add_node("anna", anna)
workflow.add_node("matteo", matteo)

# Set the entrypoint to `Anna`
workflow.set_entry_point("anna")

# We now add a conditional edge.
# We grade the greetings from matteo.
# If grade is "good greeter", we end; if not, we cycle
# back to matteo for his greetings.
workflow.add_conditional_edges(

    "matteo",
    greetings_grader,
    {
        "bad greeter": "matteo",
        "good greeter": END
    }
)

# We add an edge between `Anna` and `Matteo`
workflow.add_edge('anna', 'matteo')

app = workflow.compile()

type(app)

langgraph.pregel.Pregel

## Run Graph

In [11]:
greetings = ["Hello you two, greet politely"]
try:
    app.invoke({"keys":{"greetings": greetings}}, config={"recursion_limit": 10})
except:
    print("Oh no, you reached the greeting limit!")

System Greetings
['Hello you two, greet politely']
----Anna-----
['buongiorno', 'good morning']
------Matteo---------
['ciao', 'hi']
DECISION-----BAD GREETER----TRY AGAIN
------Matteo---------
['buongiorno', 'good morning']
DECISION-----GOOD GREETER----GOODBYE
